In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense, merge, Reshape, Permute, Multiply, Dot,dot, Concatenate, Add
from keras.layers import Input
from keras import backend as K
from keras.engine.topology import Layer
import keras as keras

# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy, CrowdsClassificationSModel, \
    CrowdsClassificationCModelSingleWeight, CrowdsClassificationCModel, MaskedMultiCrossEntropyCosSim, \
    MaskedMultiCrossEntropyBaseChannel, MaskedMultiCrossEntropyBaseChannelConst, CrowdsClassificationSModelChannelMatrix, \
    MaskedMultiCrossEntropyCurriculumChannelMatrix
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

/home/yajingyang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
# Prepare data
def load_data(filename):
    f = open(filename, 'rb')
    data = np.load(f)
    f.close()
    return data

def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

def get_data(DATA_PATH, N_CLASSES):
    
    print("\nLoading train data...")
    # images processed by VGG16
    data_train_vgg16 = load_data(DATA_PATH+"data_train_vgg16.npy")
    print(data_train_vgg16.shape)

    # ground truth labels
    labels_train = load_data(DATA_PATH+"labels_train.npy")
    print(labels_train.shape)

    # labels obtained from majority voting
    labels_train_mv = load_data(DATA_PATH+"labels_train_mv.npy")
    print(labels_train_mv.shape)

#     # labels obtained by using the approach by Dawid and Skene
#     labels_train_ds = load_data(DATA_PATH+"labels_train_DS.npy")
#     print(labels_train_ds.shape)

    # data from Amazon Mechanical Turk
    print("\nLoading AMT data...")
    answers = load_data(DATA_PATH+"answers.npy")
    print(answers.shape)
    N_ANNOT = answers.shape[1]
    print("N_CLASSES:", N_CLASSES)
    print("N_ANNOT:", N_ANNOT)

    # load test data
    print("\nLoading test data...")

    # images processed by VGG16
    data_test_vgg16 = load_data(DATA_PATH+"data_test_vgg16.npy")
    print(data_test_vgg16.shape)

    # test labels
    labels_test = load_data(DATA_PATH+"labels_test.npy")
    print(labels_test.shape)

    print("\nLoading validation data...")
    # images processed by VGG16
    data_valid_vgg16 = load_data(DATA_PATH+"data_valid_vgg16.npy")
    print(data_valid_vgg16.shape)

    # validation labels
    labels_valid = load_data(DATA_PATH+"labels_valid.npy")
    print(labels_valid.shape)

    labels_train_bin = one_hot(labels_train, N_CLASSES)
    labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
#     labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
#     print(labels_train_ds_bin.shape)
    labels_test_bin = one_hot(labels_test, N_CLASSES)
    labels_valid_bin = one_hot(labels_valid, N_CLASSES)


    answers_bin_missings = []
    for i in range(len(answers)):
        row = []
        for r in range(N_ANNOT):
            if answers[i,r] == -1:
                row.append(-1 * np.ones(N_CLASSES))
            else:
                row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
        answers_bin_missings.append(row)
    answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)

    answers_test_bin_missings = np.zeros((len(labels_test), N_CLASSES))
    answers_test_bin_missings[np.arange(len(labels_test)), labels_test] = 1
    answers_test_bin_missings = np.repeat(answers_test_bin_missings.reshape([len(labels_test),N_CLASSES,1]), N_ANNOT, axis=2)

    answers_valid_bin_missings = np.zeros((len(labels_valid), N_CLASSES))
    answers_valid_bin_missings[np.arange(len(labels_valid)), labels_valid] = 1
    answers_valid_bin_missings = np.repeat(answers_valid_bin_missings.reshape([len(labels_valid),N_CLASSES,1]), N_ANNOT, axis=2)
    
    x = {'train': data_train_vgg16, 'test': data_test_vgg16, 'val': data_valid_vgg16}
    y_gt = {'train': labels_train_bin, 'test': labels_test_bin, 'val': labels_valid_bin}
    y_annot = {'train': answers_bin_missings, 'test': answers_test_bin_missings, 'val': answers_valid_bin_missings, 'mv':labels_train_mv_bin}
    return x, y_gt, y_annot

In [4]:
# Build model
def eval(model,x_test, y_test):
    print('Test dataset results: ')
    print(dict(zip(model.metrics_names,model.evaluate(x_test,y_test, verbose=False))))


def get_trace(model):

    channel_matrix = model.get_weights()[-1]
    channel_matrix_trace = tf.trace(K.permute_dimensions(channel_matrix, [2, 0, 1]))
    channel_matrix_trace_arr = K.eval(channel_matrix_trace)
    return channel_matrix_trace_arr


def print_single_loss(model):
    import matplotlib.pyplot as plt

    # list all data in history
    print(model.history.keys())
    # summarize history for accuracy
    plt.plot(model.history['baseline_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()
    # summarize history for loss
    plt.plot(model.history['baseline_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()


def print_history(df, title):
    import matplotlib.pyplot as plt
    # Make a data frame
    df['x'] = range(df.shape[0])

    # style
    plt.style.use('seaborn-darkgrid')

    # create a color palette
    palette = plt.get_cmap('Set1')

    # multiple line plot
    num = 0
    for column in df.drop('x', axis=1):
        num += 1
        plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

    # Add legend
    plt.legend(loc=2, ncol=2)

    # Add titles
    plt.title(title, loc='left', fontsize=12, fontweight=0, color='orange')
    plt.xlabel("Time")
    plt.ylabel("Score")
    plt.savefig(title+'.png')

In [5]:
def build_base_model(train_data_shape, N_CLASSES):
    base_model = Sequential()
    base_model.add(Flatten(input_shape=train_data_shape[1:]))
    base_model.add(Dense(128, activation='relu'))
    base_model.add(Dropout(0.4))
    base_model.add(Dense(64, activation='relu'))
    base_model.add(Dropout(0.4))
    base_model.add(Dense(N_CLASSES))
    base_model.add(Activation("softmax"))
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return base_model

In [6]:
def majority_vote(x, y_gt, y_annot, N_CLASSES):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_annot['mv'], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [7]:
def baseline_gt(x, y_gt, N_CLASSES):
    train_data_shape = x['train'].shape
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_gt['train'], epochs=N_EPOCHS, shuffle=True,
                                 batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [14]:
def build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace):
    hidden_layers = Sequential()
    hidden_layers.add(Flatten(input_shape=train_data_shape[1:]))
    hidden_layers.add(Dense(128, activation='relu'))
    hidden_layers.add(Dropout(0.4))
    hidden_layers.add(Dense(64, activation='relu'))
    hidden_layers.add(Dropout(0.4))

    train_inputs = Input(shape=(train_data_shape[1:]))
    last_hidden = hidden_layers(train_inputs)
    baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

    if softmax:
        channel_layer = CrowdsClassificationSModelChannelMatrix(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer([last_hidden, baseline_output])
    else:
        channel_layer = CrowdsClassification(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer(baseline_output)



    model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

    if trace:
        loss = MaskedMultiCrossEntropyCurriculumChannelMatrix(model, 1, 1).loss
    else:
        loss = MaskedMultiCrossEntropy().loss

    # compile model with masked loss and train
    model.compile(optimizer='adam',
                         loss=[loss, 'categorical_crossentropy'],
                         loss_weights=[1, 0],
                         metrics=['accuracy']
                        )
    return model

In [15]:
def crowd_model(x, y_gt, y_annot, N_CLASSES, softmax, trace):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]

    model = build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace)    
    
    history = model.fit(x['train'], [y_annot['train'], y_gt['train']], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    trace_arr = get_trace(model)
    eval(model, x['test'], y_test=[y_annot['test'], y_gt['test']])
    return history, trace_arr

## Train with LabelMe data

In [19]:
NUM_RUNS = 20
N_CLASSES = 8
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = "/home/yajingyang/Downloads/LabelMe/prepared/"
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

# print('baseline model with majority vote labels:')
# for i in range(NUM_RUNS):
#     base_acc_df = pd.DataFrame()
#     base_loss_df = pd.DataFrame()
#     base_history = majority_vote(x, y_gt, y_annot, N_CLASSES)
#     base_acc_df.loc[:, i] = base_history.history['acc']
#     base_loss_df.loc[:, i] = base_history.history['loss']

#     base_acc_df.to_csv('base_accuracy.csv')
#     base_loss_df.to_csv('base_loss.csv')
# #     print_history(base_acc_df, 'accuracy')
# #     print_history(base_loss_df, 'loss')

# print('baseline model with clean labels')
# for i in range(NUM_RUNS):
#     clean_base_acc_df = pd.DataFrame()
#     clean_base_loss_df = pd.DataFrame()
#     clean_history = baseline_gt(x, y_gt, N_CLASSES)
#     clean_base_acc_df.loc[:, i] = clean_history.history['acc']
#     clean_base_loss_df.loc[:, i] = clean_history.history['loss']

#     clean_base_acc_df.to_csv('clean_accuracy.csv')
#     clean_base_loss_df.to_csv('clean_loss.csv')
# #     print_history(clean_acc_df, 'accuracy')
# #     print_history(clean_loss_df, 'loss')



for softmax in [False, True]:
    for trace in [False, True]:
        print('\nCrowd noise adaptation model with softmax: %s, trace: %s' % (softmax, trace))
        prefix = ''
        if softmax: 
            prefix = 'softmax_'
        if trace:
            prefix = prefix + 'trace_'
        loss_csv = prefix + 'loss.csv'
        acc_csv = prefix + 'acc.csv'
        trace_csv = prefix + 'trace.csv'
        acc_df = pd.DataFrame()
        loss_df = pd.DataFrame()
        trace_df = pd.DataFrame()
        
        for i in range(NUM_RUNS):
            history, trace_arr = crowd_model(x, y_gt, y_annot, N_CLASSES, softmax, trace)
            acc_df.loc[:, i] = history.history['baseline_acc']
            loss_df.loc[:, i] = history.history['baseline_loss']
            trace_df.loc[:, i] = trace_arr

        acc_df.to_csv(acc_csv)
        loss_df.to_csv(loss_csv)
        trace_df.to_csv(trace_csv)

#     print_history(trace_crowd_acc_df, 'accuracy')
#     print_history(trace_crowd_loss_df, 'loss')
# acc_df.loc[:, str(b)] = history.history['baseline_acc']
# loss_df.loc[:, str(b)] = history.history['baseline_loss']

# acc_df.to_csv('accuracy.csv')
# loss_df.to_csv('loss.csv')
# print_history(acc_df, 'accuracy')
# print_history(loss_df, 'loss')


Loading train data...
(10000, 4, 4, 512)
(10000,)
(10000,)

Loading AMT data...
(10000, 59)
N_CLASSES: 8
N_ANNOT: 59

Loading test data...
(1188, 4, 4, 512)
(1188,)

Loading validation data...
(500, 4, 4, 512)
(500,)

Crowd noise adaptation model with softmax: False, trace: False
Tensor("baseline_11/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_9/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_9/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
y_true:  Tensor("crowds_classification_9_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_9/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
Test dataset results: 
{'loss': 0.9748115021773059, 'crowds_classification_9_loss': 0.9748115021773059, 'baseline_loss': 2.425109326574083, 'crowds_classification_9_acc': 0.020412457912457913, 'baseline_acc': 0.7777777777777778}
Tensor("baseline_12/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classif

Test dataset results: 
{'loss': 0.9087348490050344, 'crowds_classification_21_loss': 0.9087348490050344, 'baseline_loss': 0.9182419966065923, 'crowds_classification_21_acc': 0.030303030303030304, 'baseline_acc': 0.8484848484848485}
Tensor("baseline_24/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_22/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_22/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
y_true:  Tensor("crowds_classification_22_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_22/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
Test dataset results: 
{'loss': 0.9305717415279813, 'crowds_classification_22_loss': 0.9305717415279813, 'baseline_loss': 0.9502164366896506, 'crowds_classification_22_acc': 0.032196969696969696, 'baseline_acc': 0.8417508417508418}
Tensor("baseline_25/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_23/CrowdLayer:0' shape=(8, 8, 59) d

(8,)
Test dataset results: 
{'loss': -7.069947496407763, 'crowds_classification_36_loss': -7.069947496407763, 'baseline_loss': 1.004657715257972, 'crowds_classification_36_acc': 0.030934343434343436, 'baseline_acc': 0.8375420875420876}
Tensor("baseline_39/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_37/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_37/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.067775499941122, 'crowds_classification_37_loss': -7.067775499941122, 'baseline_loss': 1.0768193518774276, 'crowds_classification_37_acc': 0.032407407407407406, 'baseline_acc': 0.8358585858585859}
Tensor("baseline_40/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_38/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_38/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.000940467372085, 'crowds_cl

y_true:  Tensor("crowds_classification_s_model_channel_matrix_7_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_s_model_channel_matrix_7/transpose:0", shape=(?, 8, 59), dtype=float32)
Test dataset results: 
{'loss': 1.61709340092309, 'crowds_classification_s_model_channel_matrix_7_loss': 1.61709340092309, 'baseline_loss': 1.81374021151702, 'crowds_classification_s_model_channel_matrix_7_acc': 0.017992424242424244, 'baseline_acc': 0.8291245791245792}
inputs:  [<tf.Tensor 'sequential_102/dropout_204/cond/Merge:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'baseline_55/Softmax:0' shape=(?, 8) dtype=float32>]
weights:  [<tf.Variable 'crowds_classification_s_model_channel_matrix_8/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>]
y_true:  Tensor("crowds_classification_s_model_channel_matrix_8_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_s_model_channel_matrix_8/transpose:0", shape=(?, 8, 59), dtype=float32)
Test dataset r

y_true:  Tensor("crowds_classification_s_model_channel_matrix_18_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_s_model_channel_matrix_18/transpose:0", shape=(?, 8, 59), dtype=float32)
Test dataset results: 
{'loss': 1.6176294437562577, 'crowds_classification_s_model_channel_matrix_18_loss': 1.6176294437562577, 'baseline_loss': 1.6682592688209883, 'crowds_classification_s_model_channel_matrix_18_acc': 0.011994949494949494, 'baseline_acc': 0.8308080808080808}
inputs:  [<tf.Tensor 'sequential_113/dropout_226/cond/Merge:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'baseline_66/Softmax:0' shape=(?, 8) dtype=float32>]
weights:  [<tf.Variable 'crowds_classification_s_model_channel_matrix_19/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>]
y_true:  Tensor("crowds_classification_s_model_channel_matrix_19_target:0", shape=(?, ?, ?), dtype=float32)
y_pred:  Tensor("crowds_classification_s_model_channel_matrix_19/transpose:0", shape=(?, 8, 59), dtype=float32)
T

(8,)
Test dataset results: 
{'loss': 6.567723169872656, 'crowds_classification_s_model_channel_matrix_31_loss': 6.567723169872656, 'baseline_loss': 3.708388747590961, 'crowds_classification_s_model_channel_matrix_31_acc': 0.014625420875420875, 'baseline_acc': 0.7196969696969697}
inputs:  [<tf.Tensor 'sequential_126/dropout_252/cond/Merge:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'baseline_79/Softmax:0' shape=(?, 8) dtype=float32>]
weights:  [<tf.Variable 'crowds_classification_s_model_channel_matrix_32/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>]
(8,)
Test dataset results: 
{'loss': 6.523539212416318, 'crowds_classification_s_model_channel_matrix_32_loss': 6.523539212416318, 'baseline_loss': 2.036656724282788, 'crowds_classification_s_model_channel_matrix_32_acc': 0.020622895622895623, 'baseline_acc': 0.8047138047138047}
inputs:  [<tf.Tensor 'sequential_127/dropout_254/cond/Merge:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'baseline_80/Softmax:0' shape=(?, 8) dtype=float32>]
we

## Train with Bird Dataset

### Cactus Wren

In [23]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Cactus Wren/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)



Loading train data...
(3069, 7, 7, 512)
(3069,)
(3069,)

Loading AMT data...
(3069, 31)
N_CLASSES: 2
N_ANNOT: 31

Loading test data...
(176, 7, 7, 512)
(176,)

Loading validation data...
(363, 7, 7, 512)
(363,)

baseline model with clean labels:
Test dataset results: 
{'loss': 4.580449364402077, 'acc': 0.6420454545454546}
Test dataset results: 
{'loss': 2.014761946418069, 'acc': 0.875}
Test dataset results: 
{'loss': 1.6465330990878018, 'acc': 0.7386363636363636}
Test dataset results: 
{'loss': 1.007381016557867, 'acc': 0.9375}

baseline model with majority vote labels:
Test dataset results: 
{'loss': 2.49463267107223, 'acc': 0.7840909090909091}
Test dataset results: 
{'loss': 2.747402613813227, 'acc': 0.8295454545454546}
Test dataset results: 
{'loss': 9.89710773121227, 'acc': 0.24431818181818182}
Test dataset results: 
{'loss': 1.8316018256274136, 'acc': 0.8863636363636364}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_54/dropout_58/cond/Merge:0' shape=(?, 128) dty

In [19]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Cape May Warbler/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)

    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)


Loading train data...
(3510, 7, 7, 512)
(3510,)
(3510,)

Loading AMT data...
(3510, 40)
N_CLASSES: 2
N_ANNOT: 40

Loading test data...
(210, 7, 7, 512)
(210,)

Loading validation data...
(405, 7, 7, 512)
(405,)

baseline model with majority vote labels:
Test dataset results: 
{'loss': 3.363896878560384, 'acc': 0.6476190487543741}
Test dataset results: 
{'loss': 3.453891637211754, 'acc': 0.7857142857142857}
Test dataset results: 
{'loss': 10.050633058093844, 'acc': 0.2857142858562015}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}

baseline model with clean labels:
Test dataset results: 
{'loss': 7.196937047867548, 'acc': 0.4190476207506089}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}
Test dataset results: 
{'loss': 5.360020560310001, 'acc': 0.5095238098076411}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_36/dropout_36/cond/Me

In [20]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Evening Grosbeak/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)

    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)


Loading train data...
(4605, 7, 7, 512)
(4605,)
(4605,)

Loading AMT data...
(4605, 26)
N_CLASSES: 2
N_ANNOT: 26

Loading test data...
(270, 7, 7, 512)
(270,)

Loading validation data...
(540, 7, 7, 512)
(540,)

baseline model with majority vote labels:
Test dataset results: 
{'loss': 5.301747682359483, 'acc': 0.5111111114422481}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}
Test dataset results: 
{'loss': 8.372794212456103, 'acc': 0.3370370357124894}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}

baseline model with clean labels:
Test dataset results: 
{'loss': 6.542580353772199, 'acc': 0.4629629620799312}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}
Test dataset results: 
{'loss': 6.45135940975613, 'acc': 0.4555555558866925}
Test dataset results: 
{'loss': 2.746045853473522, 'acc': 0.8296296296296296}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_42/dropout_42/cond/Merg